In [33]:
from langgraph.graph import StateGraph, START, END
from typing import List, TypedDict
import random

In [34]:
class AgentState(TypedDict):
    player_name: str
    guesses: List[int]
    attempts: int
    lower_bound: int
    upper_bound: int
    target_number: int
    current_guess: int
    game_over: bool
    won: bool

In [35]:
def setup_node(state: AgentState) -> AgentState:
    """Initialize the game with a random target number"""
    state['target_number'] = random.randint(1, 20)
    print(f"Game started! I'm thinking of a number between 1 and 20...")
    print(f"(Secret: The target number is {state['target_number']})")
    return state

def make_guess(state: AgentState) -> AgentState:
    """Make an intelligent guess using binary search strategy"""
    # Use binary search approach for optimal guessing
    guess = random.randint(state['lower_bound'], state['upper_bound'])
    state['current_guess'] = guess
    state['guesses'].append(guess)
    state['attempts'] += 1

    print(f"Attempt {state['attempts']}: I guess {guess}")
    return state

def hint_node(state: AgentState) -> AgentState:
    """Provide hints and update bounds based on the guess"""
    current_guess = state['current_guess']
    target = state['target_number']

    if current_guess == target:
        state['won'] = True
        state['game_over'] = True
        print(f"🎉 Correct! The number was {target}!")
        print(f"Won in {state['attempts']} attempts!")
    elif current_guess < target:
        print("Higher! 📈")
        state['lower_bound'] = current_guess + 1
    else:
        print("Lower! 📉")
        state['upper_bound'] = current_guess - 1

    # Check if we've reached max attempts
    if state['attempts'] >= 7 and not state['won']:
        state['game_over'] = True
        print(f"Game Over! Max attempts reached. The number was {target}")

    return state

def should_continue(state: AgentState) -> str:
    """Determine if the game should continue or end"""
    if state['game_over']:
        return "end"
    else:
        return "continue"

In [36]:
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("setup", setup_node)
graph.add_node("guess", make_guess)
graph.add_node("hint", hint_node)

# Set entry point
graph.set_entry_point("setup")

# Add edges
graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint")

# Add conditional edges for looping
graph.add_conditional_edges(
    "hint",
    should_continue,
    {
        "continue": "guess",
        "end": END
    }
)

app = graph.compile()



In [37]:
# Visualize the graph structure


# Test the automatic higher/lower guessing game
initial_state = {
    "player_name": "Student",
    "guesses": [],
    "attempts": 0,
    "lower_bound": 1,
    "upper_bound": 20,
    "target_number": 0,  # Will be set by setup_node
    "current_guess": 0,
    "game_over": False,
    "won": False
}

# Run the game
result = app.invoke(initial_state)

print("\n" + "="*50)
print("GAME SUMMARY:")
print("="*50)
print(f"Player: {result['player_name']}")
print(f"Target number: {result['target_number']}")
print(f"Guesses made: {result['guesses']}")
print(f"Total attempts: {result['attempts']}")
print(f"Won: {result['won']}")
if result['won']:
    print(f"🎉 Victory in {result['attempts']} attempts!")
else:
    print("😔 Better luck next time!")

Game started! I'm thinking of a number between 1 and 20...
(Secret: The target number is 2)
Attempt 1: I guess 20
Lower! 📉
Attempt 2: I guess 12
Lower! 📉
Attempt 3: I guess 11
Lower! 📉
Attempt 4: I guess 7
Lower! 📉
Attempt 5: I guess 2
🎉 Correct! The number was 2!
Won in 5 attempts!

GAME SUMMARY:
Player: Student
Target number: 2
Guesses made: [20, 12, 11, 7, 2]
Total attempts: 5
Won: True
🎉 Victory in 5 attempts!
